In [ ]:
!pip install langchain_community langchain-openai langchainhub chromadb langchain pypdf langchain_postgres psycopg2-binary python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
import os
from docx import Document
from langchain import OpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.agents import initialize_agent, Tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = 'LANG_SMITH_KEY_FOR_DEBUGGING'
os.environ['OPENAI_API_KEY'] = 'API KEY'


def load_local_doc(file_path: str) -> str:
    if file_path.endswith('.txt'):
        with open(file_path, 'r') as f:
            return f.read()
    elif file_path.endswith('.docx'):
        doc = Document(file_path)
        return "\n".join([para.text for para in doc.paragraphs])
    else:
        raise ValueError("Unsupported file format. Use .txt or .docx")

# Load the document
file_path = 'Engine overheat scenarios.docx'  # Update with your file path
document_content = load_local_doc(file_path)

# Split the document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.create_documents([document_content])

# Create embeddings using OpenAI
embeddings = OpenAIEmbeddings()

# Store embeddings in ChromaDB
vector_store = Chroma.from_documents(docs, embeddings)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
retrieval_qa_chat_prompt = hub.pull("rlm/rag-prompt")

state = {}
def simple_preprocessing(llm, new_question, state):
    retriever = vector_store.as_retriever(search_kwargs={"k": 2})
    docs = retriever.get_relevant_documents(new_question)

    # Post-processing
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    context = format_docs(docs)
    state["context"] = context

    output_parser = StrOutputParser()

    chain = retrieval_qa_chat_prompt | llm | output_parser

    return chain.invoke({"context": context, "question": new_question})

def handle_query(input_query: str) -> str:
    response = simple_preprocessing(llm, input_query, state)
    return response

tools = [
    Tool(
        name="Retrival QA",
        func=handle_query,
        description="Handle queries related to problem being faced by machine"
    )
]

# Initialize the agent with the tool
agent = initialize_agent(
    tools=tools,
    agent_type="react-docstore",
    llm=llm
)
problem = "engine is overheating"
# Define the query
query = ("give me the metrics for the problem mentioned. \n"
         "just give high level cases metrics needs to be check and what its value should be \n"
         "give response point wise \n problem: ") + "engine is overheating"


In [ ]:
# Use the agent to handle the query
response = agent.run(query)

metrics_to_check = llm.invoke("give the point wise metrics to be sent to a SQL agents to query the database \n " + response)

In [ ]:
# Output the response
print(response)
print(metrics_to_check.content)

The metrics for the engine overheating problem include EGT exceeding 1,300°F, Coolant Temperature surpassing 220°F, and minimal difference between Radiator Outlet and Inlet Temperatures.
1. EGT (Exhaust Gas Temperature) exceeding 1,300°F
2. Coolant Temperature surpassing 220°F
3. Minimal difference between Radiator Outlet and Inlet Temperatures


In [ ]:
for metric in metrics_to_check.content.split("\n"):
    print(metric)

1. EGT (Exhaust Gas Temperature) exceeding 1,300°F
2. Coolant Temperature surpassing 220°F
3. Minimal difference between Radiator Outlet and Inlet Temperatures


In [ ]:
!pip install pymysql

In [ ]:
import sqlite3
conn = sqlite3.connect('demo.db')
cursor = conn.cursor()

with open('machine_metrics.sql', 'r') as sql_file:
    sql_script = sql_file.read()

cursor.executescript(sql_script)
conn.commit()
cursor.execute('SELECT * FROM machine_running_metrics')
rows = cursor.fetchall()

for row in rows:
    print(row)
conn.close()

(1400.0, 110.0, 100.0, 250.0, '2024-09-29 00:00:00', 'abcd123')
(1000.0, 250.0, 100.0, 250.0, '2024-09-29 00:00:00', 'abcd124')
(1400.0, 110.0, 100.0, 250.0, '2024-09-29 00:00:00', 'abcd123')
(1000.0, 250.0, 100.0, 250.0, '2024-09-29 00:00:00', 'abcd124')


In [ ]:
from langchain_community.agent_toolkits import create_sql_agent
import urllib.parse
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

sqlite_uri = "sqlite:////content/demo.db"  # Ensure the correct path to your .db file

# Create the database object for SQLite
db = SQLDatabase.from_uri(sqlite_uri)
# db.run("SELECT * FROM machine_running_metrics LIMIT 10;")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)


In [ ]:
output = ""
for metric in metrics_to_check.content.split("\n"):
    result = agent_executor.invoke(metric + " for machine abcd123")
    output += "input:: " + result["input"] + "\n"
    output += "output:: " + result["output"] + "\n\n"
    agent_executor.invoke(metric + " for machine abcd123")

print(output)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


machine_running_metrics
Invoking: `sql_db_schema` with `{'table_names': 'machine_running_metrics'}`



CREATE TABLE machine_running_metrics (
	exhaust_gas_temp REAL DEFAULT NULL, 
	coolant_temp REAL DEFAULT NULL, 
	radiator_outlet_temp REAL DEFAULT NULL, 
	radiator_inlet_temp REAL DEFAULT NULL, 
	timestamp DATETIME DEFAULT NULL, 
	machine_name VARCHAR(100) DEFAULT NULL
)

/*
3 rows from machine_running_metrics table:
exhaust_gas_temp	coolant_temp	radiator_outlet_temp	radiator_inlet_temp	timestamp	machine_name
1400.0	110.0	100.0	250.0	2024-09-29 00:00:00	abcd123
1000.0	250.0	100.0	250.0	2024-09-29 00:00:00	abcd124
1400.0	110.0	100.0	250.0	2024-09-29 00:00:00	abcd123
*/
Invoking: `sql_db_query` with `{'query': "SELECT * FROM machine_running_metrics WHERE machine_name = 'abcd123' AND exhaust_gas_temp > 1300"}`


[(1400.0, 110.0, 100.0, 250.0, '2024-09-29 00:00:00', 'abcd123'), (1400.0, 110.0, 100.0, 2

In [ ]:
llm.invoke("Given the result from SQL agent and our knowledge from context. Tell us the diagnosis of the problem. \n" + "problem:: \n" + "engine is overheating" + " \n\n SQL Agent Finding:: \n" + output + "\n\n context::\n" + state["context"])

AIMessage(content='Based on the SQL Agent findings and the context provided, the diagnosis of the problem with the engine overheating for machine abcd123 is a Lean Air-Fuel Mixture or Turbocharger Issue. This is indicated by the Exhaust Gas Temperature (EGT) exceeding 1,300°F, suggesting that the engine is running with an excessively lean air-fuel ratio, leading to higher combustion temperatures and engine overheating.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 555, 'total_tokens': 638, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-20439896-3c9d-492e-82dc-8bba976e0091-0', usage_metadata={'input_tokens': 555, 'output_tokens': 83, 'total_tokens': 638})